In [1]:
import re
import itertools
import math
import time
from datetime import datetime, timezone
import collections

from bs4 import BeautifulSoup
import requests
import numpy as np
import gensim

from gensim.utils import simple_preprocess, simple_tokenize
from gensim.parsing.preprocessing import STOPWORDS, DEFAULT_FILTERS

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

import config
from events.meetup import gatherer, formatter, filters 

Using TensorFlow backend.


239 blacklist tokens


In [7]:
groups = gatherer.get_groups(config.meetup['groups_url'], config.meetup['params'].copy())
print('Gathered', len(groups), 'groups')

good_ids, multi_bad_ids, good_indexes, bad_indexes = gatherer.good_bad_group_ids(
    groups, config.multi_enhanced_blacklist_tokens, config.token_thresholds)
bad_ids = list(itertools.chain.from_iterable(multi_bad_ids))
print('Found {} good meetup groups, {} bad meetup groups'.format(len(good_ids), len(bad_ids)))
print('blockchain groups: {}, business groups: {}'.format(len(multi_bad_ids[0]), len(multi_bad_ids[1])))

events_data = gatherer.get_groups_events(config.meetup['events_url'],
    config.meetup['params'].copy(), good_ids, config.meetup['max_meetup_responses'])
print('Found {} good meetup events'.format(len(events_data)))

good_events = list(filters.get_good_events(events_data, config.blacklist_tokens,
                    config.meetup['params']['country'], config.meetup['max_event_hours']))
print('Filtered {} down to {} good meetup events'.format(len(events_data), len(good_events)))

good_filtered_events = list(filters.remove_duplicate_events(good_events))
print('Removed good duplicate events from {} to {}'.format(
    len(good_events), len(good_filtered_events)))


blockchain_events = gatherer.get_groups_events(config.meetup['events_url'],
    config.meetup['params'].copy(), multi_bad_ids[0], config.meetup['max_meetup_responses'])
print('Found {} blockchain meetup events'.format(len(blockchain_events)))
business_events = gatherer.get_groups_events(config.meetup['events_url'],
    config.meetup['params'].copy(), multi_bad_ids[1], config.meetup['max_meetup_responses'])
print('Found {} business meetup events'.format(len(business_events)))

Gathered 548 groups
Found 403 good meetup groups, 145 bad meetup groups
blockchain groups: 118, business groups: 27
Found 247 good meetup events
Filtered 247 down to 131 good meetup events
Removed good duplicate events from 131 to 69
Found 317 blockchain meetup events
Found 197 business meetup events


In [10]:
blockchain_events = list(filters.remove_duplicate_events(blockchain_events))
print(len(blockchain_events))

35


In [12]:
business_events = list(filters.remove_duplicate_events(business_events))
print(len(business_events))

11


In [53]:
good_tokens = [list(filters.tokenize(e['description'])) for e in good_filtered_events]
blockchain_tokens = [list(filters.tokenize(e['description'])) for e in blockchain_events]
business_tokens = [list(filters.tokenize(e['description'])) for e in business_events]

In [54]:
c = collections.Counter(itertools.chain.from_iterable(blockchain_tokens))
c.update(itertools.chain.from_iterable(business_tokens))
c.update(itertools.chain.from_iterable(good_tokens))
vocab = set(list(c.elements()))
len(vocab)

2674

In [55]:
index_to_word = {i: word for i, word in enumerate(vocab)}
word_to_index = {word: i for i, word in enumerate(vocab)}

In [56]:
blockchain_sentences = []
for tokens in blockchain_tokens:
    blockchain_sentences.append([word_to_index[word] for word in tokens])
    
business_sentences = []
for tokens in business_tokens:
    business_sentences.append([word_to_index[word] for word in tokens])
    
good_sentences = []
for tokens in good_tokens:
    good_sentences.append([word_to_index[word] for word in tokens])

In [58]:
len(blockchain_sentences), len(business_sentences), len(good_sentences)

(35, 11, 69)

In [80]:
x_train = sequence.pad_sequences(good_sentences[:11] + blockchain_sentences[:11] + business_sentences)

In [101]:
y_train = np.zeros((33, 3))
y_train[:11, 0] = 1
y_train[11:22, 1] = 1
y_train[22:, 2] = 1

In [102]:
x_train.shape, y_train.shape

((33, 197), (33, 3))

In [103]:
epochs = 100


# Embedding
max_features = len(vocab)
input_length = x_train.shape[1]
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=input_length))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          epochs=epochs,
          validation_split=0.1)

Build model...
Train...
Train on 29 samples, validate on 4 samples
Epoch 1/100
29/29 [==============================] - 1s 47ms/step - loss: 0.6942 - acc: 0.4828 - val_loss: 0.7005 - val_acc: 0.3333
Epoch 2/100
29/29 [==============================] - 0s 4ms/step - loss: 0.6863 - acc: 0.6092 - val_loss: 0.7025 - val_acc: 0.3333
Epoch 3/100
29/29 [==============================] - 0s 4ms/step - loss: 0.6782 - acc: 0.7356 - val_loss: 0.7050 - val_acc: 0.5833
Epoch 4/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6705 - acc: 0.7586 - val_loss: 0.7086 - val_acc: 0.5833
Epoch 5/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6611 - acc: 0.7126 - val_loss: 0.7134 - val_acc: 0.5833
Epoch 6/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6517 - acc: 0.7011 - val_loss: 0.7203 - val_acc: 0.5833
Epoch 7/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6407 - acc: 0.6897 - val_loss: 0.7302 - val_acc: 0.5833
Epoch 8/100
29/

Epoch 63/100
29/29 [==============================] - 0s 4ms/step - loss: 0.0096 - acc: 1.0000 - val_loss: 2.7892 - val_acc: 0.3333
Epoch 64/100
29/29 [==============================] - 0s 4ms/step - loss: 0.0091 - acc: 1.0000 - val_loss: 2.8250 - val_acc: 0.3333
Epoch 65/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0086 - acc: 1.0000 - val_loss: 2.8577 - val_acc: 0.3333
Epoch 66/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0081 - acc: 1.0000 - val_loss: 2.8883 - val_acc: 0.3333
Epoch 67/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0078 - acc: 1.0000 - val_loss: 2.9167 - val_acc: 0.3333
Epoch 68/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0075 - acc: 1.0000 - val_loss: 2.9427 - val_acc: 0.3333
Epoch 69/100
29/29 [==============================] - 0s 3ms/step - loss: 0.0071 - acc: 1.0000 - val_loss: 2.9671 - val_acc: 0.3333
Epoch 70/100
29/29 [==============================] - 0s 4ms/step - loss: 0.

In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [112]:
x_test = sequence.pad_sequences(good_sentences[11:], maxlen=input_length)
pred = model.predict(x_test)
good_filtered_events[-3]

{'created': 1522319644000,
 'description': '<p>There are 8 ways according to "the art of user story writing". This series will be split into 2. Each session will cover 4 ways of user story slicing.</p>',
 'duration': 5400000,
 'event_url': 'https://www.meetup.com/Scrum-Master-Guild/events/249222554/',
 'group': {'created': 1521545099000,
  'group_lat': 1.2999999523162842,
  'group_lon': 103.8499984741211,
  'id': 27886025,
  'join_mode': 'open',
  'name': 'Scrum Master Guild',
  'urlname': 'Scrum-Master-Guild',
  'who': 'Members'},
 'headcount': 0,
 'id': '249222554',
 'maybe_rsvp_count': 0,
 'name': 'Topic - Story Slicing - 1',
 'status': 'upcoming',
 'time': 1530009000000,
 'updated': 1522319644000,
 'utc_offset': 28800000,
 'venue': {'address_1': '10 Marina Boulevard, Singapore 018983',
  'city': 'Singapore',
  'country': 'sg',
  'id': 25036122,
  'lat': 1.279246,
  'localized_country_name': 'Singapore',
  'lon': 103.854111,
  'name': 'IBM Studio, MBFC Tower 2, Level 42',
  'repinne